In [14]:
import pandas as pd
import numpy as np
import pdfplumber

from IPython import display

# American Time Use Survey (ATUS) Data Dictionary

Parse column name and description from the tables in the PDF documentation.

In [3]:
pdf = pdfplumber.open("data/Documentation/atusintcodebk0320.pdf")

In [40]:
page_tables = []
for page in pdf.pages[11:54]:    # pp. 12-54
    tables = page.extract_tables()
    page_tables.append(tables)

In [41]:
# check first, second, and third level length
# corresponds to #tables, #rows, #columns
for t in page_tables:
    print(f"{len(t):3d} {len(t[0]):3d} {len(t[0][0]):3d}")

  1  28   6
  1   9   6
  1  34   6
  1  27   6
  1  31   6
  1  27   6
  1  21   6
  1  31   6
  1  30   6
  1  31   6
  1  31   6
  1  23   6
  1  27   6
  1  28   6
  1  17   6
  1  31   6
  1  31   6
  1  25   6
  1  23   6
  1  16   6
  1  21   6
  1  21   6
  1  26   6
  1  28   6
  1  26   6
  1  26   6
  1  27   6
  1  30   6
  1  33   6
  1  32   6
  1  29   6
  1  25   6
  1  24   6
  1  27   6
  1  32   6
  1  23   6
  1  25   6
  1  33   6
  1  34   6
  1  30   6
  1  14   6
  1  30   6
  1   4   6


In [42]:
parsed = []
for t in page_tables:
    t0 = t[0]

    start_of_coldesc = [row[0] == 'Name' for row in t0]
    idx = np.concatenate((np.where(start_of_coldesc)[0], [len(t0)]))
    chunks = [t0[i1:i2] for i1, i2 in zip(idx[:-1],idx[1:])]
    
    for one_col_desc in chunks:
        assert one_col_desc[0] == ['Name', 'Description', None, None, None, 'File']
        clean_desc = {
            'name': one_col_desc[1][0],
            'description': one_col_desc[1][1].replace('\n', ' '),
            'file': one_col_desc[1][5].replace('\n', ' ').split(', '),
        }
        if len(one_col_desc) > 2:
            if one_col_desc[2][1].startswith("Edited"):
                clean_desc['edited universe'] = one_col_desc[2][2]
        assert (one_col_desc[1][2:5] == [None, None, '']) or (one_col_desc[1][2:5] == [None, None, None])
        parsed.append(clean_desc)

In [43]:
parsed = pd.DataFrame(parsed)

In [44]:
col = parsed['file']
a = []
for x in list(col.values):
    a.extend(x)

pd.Series(a).value_counts()

Respondent File          132
Activity File             28
Activity Summary File     19
EC Roster File             7
Who File                   4
Roster File                4
ATUS-CPS File              1
All Files                  1
EH Activity File           1
dtype: int64

## Activity File

In [45]:
columns = pd.read_csv("data/atusact-0320/atusact_0320.dat", nrows=1).columns

in_activityfile = parsed['file'].apply(lambda x: 'Activity File' in x or 'All Files' in x)

with pd.option_context("display.max_colwidth", 200):
    selected = parsed.loc[in_activityfile, ['name','description']]
    assert set(d) == set(selected['name'].values)
    display.display(selected.set_index('name').loc[d]
                    .style.set_properties(**{'text-align': 'left'}))

,description
TUCASEID,ATUS Case ID (14-digit identifier)
TUACTIVITY_N,Activity line number
TUACTDUR24,Duration of activity in minutes (last activity truncated at 4:00 a.m.)
TUCC5,Was at least one of your own household children < 13 in your care during this activity?
TUCC5B,Was at least one of your non-own household children < 13 in your care during this activity?
TRTCCTOT_LN,Total time spent during activity providing secondary childcare for all children < 13 (in minutes)
TRTCC_LN,Total time spent during activity providing secondary child care for household and own nonhousehold children < 13 (in minutes)
TRTCOC_LN,"Total time spent during activity providing secondary child care for nonown, nonhousehold children <13 (in minutes)"
TUSTARTTIM,Activity start time
TUSTOPTIME,Activity stop time


## Activity summary file

- contains 431 `t<6 digit code>` columns.

In [202]:
columns = pd.read_csv("data/atussum-0320/atussum_0320.dat", nrows=1).columns

print("number of t?????? columns =", len(columns[columns.str.startswith('t')].str[1:]))
columns = columns[~columns.str.startswith('t')]
in_activitysumfile = parsed['file'].apply(lambda x: 'Activity Summary File' in x or 'All Files' in x)

with pd.option_context("display.max_colwidth", 200):
    selected = parsed.loc[in_activitysumfile, ['name','description']]
    try:
        assert set(columns) == set(selected['name'].values)
    except AssertionError:
        print("columns not in the Data Book", end=': ')
        print(set(columns) - set(selected['name'].values))
    display.display(selected.set_index('name').loc[[x for x in columns if x in selected['name'].values]]
                    .style.set_properties(**{'text-align': 'left'}))

number of t?????? columns = 431
columns not in the Data Book: {'PEEDUCA', 'GEMETSTA', 'PEHSPNON', 'GTMETSTA', 'PTDTRACE'}


,description
name,
TUCASEID,ATUS Case ID (14-digit identifier)
TEAGE,Edited: age
TELFS,Edited: labor force status
TEMJOT,Edited: in the last seven days did you have more than one job?
TESCHENR,"Edited: are you enrolled in high school, college, or university?"
TESCHLVL,"Edited: would that be high school, college, or university?"
TESEX,Edited: sex
TESPEMPNOT,Edited: employment status of spouse or unmarried partner
TRCHILDNUM,Number of household children < 18


## Respondent File

In [48]:
columns = pd.read_csv('data/atusresp-0320/atusresp_0320.dat', nrows=1).columns

in_respfile = parsed['file'].apply(lambda x: 'Respondent File' in x or 'All Files' in x)

with pd.option_context("display.max_colwidth", 200):
    selected = parsed.loc[in_respfile, ['name','description']]
    assert set(columns) == set(selected['name'].values)
    display.display(selected.set_index('name').loc[columns]
                    .style.set_properties(**{'text-align': 'left'}))

,description
TUCASEID,ATUS Case ID (14-digit identifier)
TULINENO,ATUS person line number
TESPUHRS,Edited: usual hours of work per week of spouse or unmarried partner
TRDTIND1,Detailed industry recode (main job)
TRDTOCC1,Detailed occupation recode (main job)
TRERNHLY,Hourly earnings at main job (2 implied decimals)
TRERNUPD,Earnings update flag
TRHERNAL,TRERNHLY: allocation flag
TRHHCHILD,Presence of household children < 18
TRIMIND1,Intermediate industry recode (main job)


## Roster File

In [165]:
columns = pd.read_csv('data/atusrost-0320/atusrost_0320.dat', nrows=1).columns

in_rostfile = parsed['file'].apply(lambda x: 'Roster File' in x or 'All Files' in x)

with pd.option_context("display.max_colwidth", 200):
    selected = parsed.loc[in_rostfile, ['name','description']]
    assert set(columns) == set(selected['name'].values)
    display.display(selected.set_index('name').loc[columns]
                    .style.set_properties(**{'text-align': 'left'}))

,description
TUCASEID,ATUS Case ID (14-digit identifier)
TULINENO,ATUS person line number
TERRP,Edited: how is this person related to you?
TEAGE,Edited: age
TESEX,Edited: sex


# Activity Lexicon

In [49]:
pdf = pdfplumber.open("data/Documentation/lexiconnoex0320.pdf")

In [51]:
page_tables = []
for page in pdf.pages[1:]:    # pp. 12-54
    tables = page.extract_tables()
    page_tables.append(tables)

In [52]:
# check first, second, and third level length
# corresponds to #tables, #rows, #columns
for t in page_tables:
    print(f"{len(t):3d} {len(t[0]):3d} {len(t[0][0]):3d}")

  1  62   5
  1  64   5
  1  63   5
  1  62   5
  1  62   5
  1  64   5
  1  64   5
  1  54   5
  1  58   5
  1  23   5


In [65]:
# exclude first two rows on each page as they are repeated headers
dfs = [pd.DataFrame(t[0][2:]) for t in page_tables]
merged_df = pd.concat(dfs)

In [134]:
rows = []
current_major_category = None
first_second_tier_category = None
for t in page_tables:
    table = t[0][2:]
    for r in table:
        if r[0]:
            current_major_category = r[0]
#             rows.append([current_major_category,,,,])
        else:
            if r[1]:
                # HACK: to fix a couple of problematic rows
                r[1] = r[1].replace('0 808 S ecurity', '0808 Security')
                first_second_tier_category = r[1]
#                 rows.append([current_major_category, first_second_tier_category,,,])
            else:
                rows.append([current_major_category, first_second_tier_category] + r[2:])

In [136]:
lexicondf = pd.DataFrame(rows, columns=['Major','FirstSecondTier','6digit','Activity','Notes'])

In [150]:
lexicondf['MajorCode'] = lexicondf['Major'].str[:2]
lexicondf['FSTierCode'] = lexicondf['FirstSecondTier'].str[:4]
lexicondf['MajorDesc'] = lexicondf['Major'].str[3:].str.strip()

In [153]:
# sanity consistency check
assert (lexicondf['6digit'].str[:4] == lexicondf['FSTierCode']).all()

Number of subcategories for each major category:

In [175]:
lexicondf

,Major,FirstSecondTier,6digit,Activity,Notes,MajorCode,FSTierCode,MajorDesc
0,01 Personal Care Activities,0101 Sleeping,010101,Sleeping,,01,0101,Personal Care Activities
1,01 Personal Care Activities,0101 Sleeping,010102,Sleeplessness,,01,0101,Personal Care Activities
2,01 Personal Care Activities,0101 Sleeping,010199,"Sleeping, n.e.c.*",,01,0101,Personal Care Activities
3,01 Personal Care Activities,0102 Grooming,010201,"Washing, dressing and grooming oneself",,01,0102,Personal Care Activities
4,01 Personal Care Activities,0102 Grooming,010299,"Grooming, n.e.c.*",,01,0102,Personal Care Activities
...,...,...,...,...,...,...,...,...
426,50 Data Codes,5001 Unable to Code,500104,Recorded simultaneous activities incorrectly,code was used in 2003-2012 only,50,5001,Data Codes
427,50 Data Codes,5001 Unable to Code,500105,"Respondent refused to provide information/""non...",,50,5001,Data Codes
428,50 Data Codes,5001 Unable to Code,500106,Gap/can't remember,,50,5001,Data Codes
429,50 Data Codes,5001 Unable to Code,500107,Unable to code activity at 1st tier,,50,5001,Data Codes


In [159]:
lexicondf["Major"].value_counts()

13 Sports, Exercise, & Recreation                       77
18 Traveling                                            53
04 Caring For & Helping Nonhousehold (NonHH) Members    37
03 Caring For & Helping Household (HH) Members          33
02 Household Activities                                 32
12 Socializing, Relaxing, and Leisure                   31
08 Professional & Personal Care Services                26
15 Volunteer Activities                                 24
05 Work & Work-Related Activities                       20
09 Household Services                                   18
06 Education                                            18
01 Personal Care Activities                             12
10 Government Services & Civic Obligations              12
07 Consumer Purchases                                   11
16 Telephone Calls                                       9
50  Data Codes                                           7
14 Religious and Spiritual Activities                   

In [160]:
%store lexicondf

Stored 'lexicondf' (DataFrame)
